<a href="https://colab.research.google.com/github/Ismail-Armutcu/Algorithms-for-Interactive-Sytems/blob/main/MMI513_PA4_Tournaments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Programming Assignment #4**

## Fall 2023/24 Semester

1. Your professor, while looking back at how badly he implemented dictionary structure that holds the player data feels frustrated. He thinks it may just be much easier to implement a `Player` class with appropriate variables (`name`, `rank`, `peerWinner`), the getter and setter functions (`getrank`, `setrank` etc.) and perhaps other utilities like `copy` Your first task is to implement this class.

2. You will then implement `unrankedplayers()`, `initialrankadjustment()` and `pyramidmatch()` functions.

3. Finally, you will implement `kingofthehill()` tournament given on p. 112 (Algorithm 5.5) of Jouni and Smed.

**GRADING:**
- `Player` class is implemented (5/100)
- `unrankedplayers()` using `Player` class (5/100)
- `initialrankadjustment()` using `Player` class (5/100)
- `pyramidmatch()` using the `Player` class (5/100)
- `kingofthehill()` function works with default arguments (40/100)
- Code carries out necessary checks on inputs via proper assertions (20/100)
- Code is properly commented, explained and documented (20/100)


**Notes:**

- You are free to use chatGPT for help. However, you should not use code generated by chatGPT directly and you should mention which part of the code is chatGPT in the comments. Your code will be checked for verbatim copying from chatGPT.
- You are not allowed to use external modules/libraries in your implementations except those already in the notebook, but you can use them for testing your code.
- You will upload your submission as a Python notebook to ODTUClass.

In [498]:
import numpy as np
import math

In [499]:
class Player:

  def __init__(self, name, rank = 0,peerStatus = True):
    assert(type(peerStatus) == bool)
    assert(rank == None or rank >=0)
    self.name = name
    self.rank = rank
    self.peerWinner = peerStatus

  def __str__(self):
    return f"Name:{self.name} Rank:{self.rank} PeerWinner:{self.peerWinner}"
  def __repr__(self):
    return str(self)
  def __eq__(self, object2):
    if(type(object2) == Player):
      return self.name == object2.name
    elif(type(object2) == str):
      return self.name == object2
    else:
      return False
  def __ne__(self, object2):
    return self.name != object2.name

  def setRank(self,rank):
    assert( rank >=0 or rank == None)
    self.rank = rank

  def getRank(self):
    return self.rank

  def setPeerWinner(self, status):
    assert(type(status) == bool)
    self.peerWinner = status

  def getPeerWinner(self):
    return self.peerWinner



def printPlayers(P):
  names = []
  ranks = []
  peerWinner = []
  for p in P:
    names.append(p.name)
    ranks.append(p.getRank())
    peerWinner.append(p.getPeerWinner())
  print("Names:     ",names)
  print("Ranks:     ",ranks)
  print("PeerWinner:",peerWinner)


In [500]:
def unrankedplayers(n):
    P = []
    for ind in range(n):
        p = Player(name = 'P' + str(ind), rank = None, peerStatus = True)
        P.append(p)

    return P

def enumeration(P):
    lst = np.arange(len(P))
    order = np.random.permutation(lst)
    M = []

    for ind in order:
      M.append(P[ind])

    return M

def rankeds(P, r):
    W = []
    for p in P:
      if p.getRank() == r:
        W.append(p)
    return W

def setrank(P, index, r):
    R = P.copy()
    R[index].setRank(r)
    return R

def match(p, q, thr, tieflag=False):
    toss = np.random.rand(1)
    upC = 1
    downC = 1
    if tieflag: # If Tie is allowed
      upC = 1.1
      downC = 0.9
    if thr * downC < toss < thr * upC: # If the random number is around thr, result is a tie
        result = Player("None",None,False) # Tie
    elif toss > thr * upC:
        result = p
    else:
        result = q
    return result

def ladder_match(P, p, q):
    m = match(p, q, 0.5)
    if m == None or m == p:
        return P
    else:
        indp = P.index(p)
        indq = P.index(q)
        rankp = P[indp].getRank()
        rankq = P[indq].getRank()
        if rankq == None:
            rankq = rankp
        P[indp].setRank(rankq)
        P[indq].setRank(rankp)
        return P

def initial_rank_adjustment(P, S):
    assert len(P) == sum(S)
    m = len(S)
    R = P.copy()
    R = enumeration(R)
    for ind in range(S[0]):
      R[ind].setRank(0) # The initial champion(s)
    c =  S[0]
    for rank in range(1, m):
        W = rankeds(R, rank - 1) # The runners-up
        Mp = enumeration(W)
        for ind in range(S[rank]):
            R[c+ind].setRank(rank)
            jnd = ind % len(Mp)

            if Mp[jnd].getRank() != rank:
              R = ladder_match(R, Mp[jnd], R[c + ind])
        c += S[rank]
    return R

In [501]:
P = unrankedplayers(11)
printPlayers(P)



S = [1, 2, 3, 5] # Ranking structure: One champion, 2 second places, etc.
M = enumeration(P)
printPlayers(M)
R = initial_rank_adjustment(P, S)
printPlayers(R)



Names:      ['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10']
Ranks:      [None, None, None, None, None, None, None, None, None, None, None]
PeerWinner: [True, True, True, True, True, True, True, True, True, True, True]
Names:      ['P10', 'P8', 'P1', 'P3', 'P9', 'P7', 'P6', 'P4', 'P2', 'P0', 'P5']
Ranks:      [None, None, None, None, None, None, None, None, None, None, None]
PeerWinner: [True, True, True, True, True, True, True, True, True, True, True]
Names:      ['P8', 'P7', 'P5', 'P4', 'P6', 'P1', 'P0', 'P2', 'P3', 'P9', 'P10']
Ranks:      [2, 0, 3, 3, 1, 1, 2, 3, 2, 3, 3]
PeerWinner: [True, True, True, True, True, True, True, True, True, True, True]


In [502]:
def pyramid_match(P, p, q):

  pi = P.index(p)
  qi = P.index(q)
  playerP = P[pi]
  playerQ = P[qi]

  #print((playerP.getRank() == playerQ.getRank()) and (not playerQ.getPeerWinner()))
  #print((playerP.getRank() == playerQ.getRank()-1) and playerQ.getPeerWinner())




  if not ((playerP.getRank() == playerQ.getRank()) and (not playerQ.getPeerWinner())) | ((playerP.getRank() == playerQ.getRank()-1) and playerQ.getPeerWinner()):
    print('No match was played! Ranks do not match the conditions for pyramid match!')
    return P

  R = P.copy() # We copy the player dictionary first
  R.pop(R.index(playerP))
  R.pop(R.index(playerQ))

  #printPlayers(R)

  # We do not have the two players in our set of players now.
  m = match(playerP, playerQ, 0.5) # Let them compete

  if (playerP.getRank() == playerQ.getRank()): # Peer match
    if (((m == p) and (not playerP.getPeerWinner())) or (((m == q) or m == "None") and playerP.getPeerWinner())):
      playerPP = Player(playerP.name,playerP.getRank(),playerP == m)
    else:
      playerPP = playerP
    if (m == q):
      playerQP = Player(playerQ.name, playerQ.getRank(), True)
    else:
       playerQP = playerQ

    R.append(playerPP)
    R.append(playerQP)
    return R
  else: # Rank challenge match
    playerQP = Player(playerQ.name, playerQ.getRank(), False)
    if (m == p) or m == "None":
      R.append(playerP)
      R.append(playerQP)
      return R
    else:
      playerPP = Player(playerP.name, playerQ.getRank(), False)
      playerQP.setRank(playerP.getRank())
      R.append(playerPP)
      R.append(playerQP)
      return R

In [503]:
n = 11
P = unrankedplayers(n)
S = [1, 2, 3, 5]
R = initial_rank_adjustment(P, S)
# Appending the peerWinner field to R makes it into pyramid structure
for r in R:
  r.setPeerWinner(True)
printPlayers(R)

Names:      ['P8', 'P1', 'P7', 'P2', 'P10', 'P4', 'P0', 'P6', 'P5', 'P3', 'P9']
Ranks:      [1, 0, 2, 1, 2, 2, 3, 3, 3, 3, 3]
PeerWinner: [True, True, True, True, True, True, True, True, True, True, True]


In [504]:
R = pyramid_match(R, "P0", "P1")
printPlayers(R)

No match was played! Ranks do not match the conditions for pyramid match!
Names:      ['P8', 'P1', 'P7', 'P2', 'P10', 'P4', 'P0', 'P6', 'P5', 'P3', 'P9']
Ranks:      [1, 0, 2, 1, 2, 2, 3, 3, 3, 3, 3]
PeerWinner: [True, True, True, True, True, True, True, True, True, True, True]


In [505]:
def kingofthehill(P = unrankedplayers(7)):
  assert(len(P) >= 1)
  assert(math.log2(len(P)+1).is_integer())
  assert(type(P[0]) == Player)

  m = int(math.log2(len(P)+1))
  S = []
  for i in range(m):
    S.append(2**i)

  R = initial_rank_adjustment(P,S)
  printPlayers(R)
  for rp in range(1,m):
    r = (m-1)-(rp-1)
    M = enumeration(rankeds(R,r))
    l = len(M)

    for i in range(l//2):
      M[2*i].setPeerWinner(False)
      M[2*i+1].setPeerWinner(False)
      print("Players face off before")
      printPlayers([M[2*i],  M[2*i+1]])
      R = pyramid_match(R,M[2*i].name,M[2*i+1].name)
      print("Players face off after")
      ind1 = R.index(M[2*i])
      ind2 = R.index(M[2*i+1])
      printPlayers([R[ind1],  R[ind2]])

    M = []
    for p in rankeds(R,r):
      if(p.getPeerWinner() == True):
        M.append(p)
    Mp = enumeration(rankeds(R,r-1))
    for i in range(l//2):
      print("Players face off before")
      printPlayers([Mp[i],M[i]])
      R = pyramid_match(R,Mp[i].name,M[i].name)
      print("Players face off after")
      ind1 = R.index(Mp[i])
      ind2 = R.index(M[i])
      printPlayers([R[ind1],  R[ind2]])

  return R


In [506]:
R = kingofthehill()
printPlayers(R)

Names:      ['P2', 'P6', 'P1', 'P0', 'P5', 'P4', 'P3']
Ranks:      [0, 2, 2, 1, 1, 2, 2]
PeerWinner: [True, True, True, True, True, True, True]
Players face off before
Names:      ['P1', 'P4']
Ranks:      [2, 2]
PeerWinner: [False, False]
Players face off after
Names:      ['P1', 'P4']
Ranks:      [2, 2]
PeerWinner: [True, False]
Players face off before
Names:      ['P3', 'P6']
Ranks:      [2, 2]
PeerWinner: [False, False]
Players face off after
Names:      ['P3', 'P6']
Ranks:      [2, 2]
PeerWinner: [False, True]
Players face off before
Names:      ['P5', 'P1']
Ranks:      [1, 2]
PeerWinner: [True, True]
Players face off after
Names:      ['P5', 'P1']
Ranks:      [1, 2]
PeerWinner: [True, False]
Players face off before
Names:      ['P0', 'P6']
Ranks:      [1, 2]
PeerWinner: [True, True]
Players face off after
Names:      ['P0', 'P6']
Ranks:      [2, 1]
PeerWinner: [False, False]
Players face off before
Names:      ['P6', 'P5']
Ranks:      [1, 1]
PeerWinner: [False, False]
Players face